In [1]:
import os 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt


%matplotlib inline 
import warnings
warnings.filterwarnings("ignore")


import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras.models import Sequential, Model 
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.layers import GlobalMaxPool2D, Dropout, SpatialDropout2D
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import load_img, array_to_img, img_to_array
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.model_selection import train_test_split

In [2]:
# warnings TensorFlow 
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

<hr>
<br>

##### Using Pre-trained Network 

In [3]:
# define model 
shape = Input(shape=(150, 150, 3))
model = ResNet50(include_top=True, weights="imagenet", input_tensor=shape)

In [4]:
# pre-process image 

path = "mountain.jpg"

img = load_img(path=path, target_size=(150, 150, 3))
img = img_to_array(img)
img = np.expand_dims(img, axis=0)
x = preprocess_input(img)

In [5]:
pred = model.predict(x)
print("Predicted: ", decode_predictions(pred, top=3)[0])

Predicted:  [('n09193705', 'alp', 0.8854446), ('n03792972', 'mountain_tent', 0.061749563), ('n09332890', 'lakeside', 0.009889168)]


<hr>
<br>

##### Extract Features with VGG-16 


In [6]:
# VGG-16 
base_model = VGG16(include_top=False, weights="imagenet")

In [7]:
# Feature extractor VGG-16 
model = Model(inputs=base_model.inputs, outputs=base_model.get_layer('block4_pool').output)

In [8]:
path = "mountain.jpg"

img = load_img(path=path, target_size=(150, 150, 3))
img = img_to_array(img)
img = np.expand_dims(img, axis=0)
x = preprocess_input(img)

In [9]:
# features 
features = model.predict(x)

<hr>
<br>

##### Fine-Tuning


In [10]:
images = ["mountain.jpg", "valey.jpg"]
labels = [0, 1]

In [11]:
new_images = []

for img in images: 
  img = load_img(img, color_mode="rgb")
  img = img_to_array(img)
# img = np.expand_dims(img, axis=0)
  img /= 255.0
  new_images.append(img)

In [12]:
data = pd.DataFrame({"image":new_images, "label":labels})

X = np.array(new_images)
y = np.array(data["label"])

data.head()

,image,label
0,"[[[0.8156863, 0.827451, 0.84705883], [0.815686...",0
1,"[[[0.5921569, 0.8039216, 0.9882353], [0.596078...",1


In [13]:
X.shape

(2, 150, 150, 3)

In [14]:
X_train, _, y_train, _ = train_test_split(X,y, random_state=42)

In [15]:
X_train.shape

(1, 150, 150, 3)

In [16]:
# freeze all layers 
vgg = VGG16(include_top=False, weights="imagenet", input_shape=(150,150,3))

for layer in vgg.layers:
  layer.trainable = False

In [17]:
# Build CNN fine-tuning 

vgg16 = vgg.output
x = Flatten()(vgg16)
x = Dense(64, activation="relu")(x)
output_layer = Dense(10, activation="softmax")(x)

model = Model(inputs=vgg.input, outputs=output_layer)
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)      

In [18]:
model.compile(optimizer=Adam(0.001), 
              loss=SparseCategoricalCrossentropy(),
              metrics=["accuracy"])

In [19]:
model.fit(X_train, y_train, epochs=10, batch_size=1)

Epoch 1/10
1/1 [==============================] - 0s 2ms/step - loss: 1.6428 - accuracy: 1.0000
Epoch 2/10
1/1 [==============================] - 0s 1ms/step - loss: 0.1726 - accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - 0s 2ms/step - loss: 0.0250 - accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 0s 3ms/step - loss: 0.0052 - accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 0s 1ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 5ms/step - loss: 4.5468e-04 - accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 1ms/step - loss: 1.7105e-04 - accuracy: 1.0000
Epoch 8/10
1/1 [==============================] - 0s 1ms/step - loss: 7.2238e-05 - accuracy: 1.0000
Epoch 9/10
1/1 [==============================] - 0s 1ms/step - loss: 3.3736e-05 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 0s 1ms/step - loss: 1.7047e-05 - accuracy: 1.0000
